In [1]:
#do some imports

import os
import sys
import enum
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import numpy as np
import pickle
import napari
from magicgui import magicgui
from qtpy.QtCore import Qt

#%run D:\\Segment\\Scripts\\newFunctions.py
from matplotlib.backends.backend_qt5agg import FigureCanvas

base_dir = 'D:\\Segment'
project_name = 'WT_01_05_19'

path = os.path.join(base_dir, project_name)
%gui qt



In [2]:
#load the labels and the dictionary containing label dtaa

labs = np.load(os.path.join(path, 'labs', 'cell_labs.npz'))
labs = labs['arr_0']

with open (os.path.join(path, 'dicts', 'cell_dict'), 'rb') as fp:
    cell_dict = pickle.load(fp)


In [ ]:
#initialize napari viewer
viewer = napari.Viewer()

#generate a matplotlib plot
plot_fig = plt.figure()
ax = plot_fig.add_subplot(1, 1, 1)

#dock the plot to the napari window
viewer.window.add_dock_widget(FigureCanvas(plot_fig), area="bottom")

#add the labels
layer = viewer.add_labels(labs.astype(int), name='labs')

#initialize the plot
(myPlot,) = ax.plot(0, 0, linewidth=5)
(smoothed,) = ax.plot(0, 0, color='r', alpha = 0.5, linestyle='--', linewidth=5)
myPlot.figure.canvas.draw()
smoothed.figure.canvas.draw()

#set axis limits and colors
axes = plt.gca()
#axes.set_xlim(np.min(timepoints), np.max(timepoints))
#axes.set_ylim(0, np.max(area) * (1.15))
axes.set_xlabel('time')
axes.set_ylabel('volume (px  )')
axes.spines['bottom'].set_color('white')
axes.xaxis.label.set_color('white')
axes.tick_params(axis='x', colors='white')
axes.spines['left'].set_color('white')
axes.yaxis.label.set_color('white')
axes.tick_params(axis='y', colors='white')


#update the plot on mouse click
@layer.mouse_drag_callbacks.append
def update_layer(layer, event):
    im = viewer.layers['labs'].data
    #get the click position in world coordinates and convert those to image coordinates
    pos = viewer.cursor.position
    pos_layer = layer.world_to_data(pos)
    #get the value of the label at that image coordinate
    value = im[pos_layer[0].astype(int), pos_layer[1].astype(int), pos_layer[2].astype(int), pos_layer[3].astype(int)]
    #pass this value to GetVolume
    try:
        GetVolume(value, smooth=True)
    except:
        ValueError

        
#function to get the volume over time from a dict and update the plot
@magicgui(call_button='Get Volume')
def GetVolume(cell:int=0, smooth:bool=False):
    
    #make a list of the timepoints and volumes to plot
    area = []
    timepoints = []
    for tp in cell_dict.keys():
        if cell in cell_dict[tp].keys():
            area.append(cell_dict[tp][cell]['Area'])
            timepoints.append(tp)
    
    #pass these to the plot
    myPlot.set_data(timepoints, area)
    #rescale the axes
    axes.set_xlim(np.min(timepoints), np.max(timepoints))
    axes.set_ylim(0, np.max(area) * (1.15))
    #draw the plot
    myPlot.figure.canvas.draw()

    #add a smoothed line, if wanted
    if smooth:
        area_filtered = savgol_filter(area, 11, 3)
        smoothed.set_data(timepoints, area_filtered)
        smoothed.figure.canvas.draw()

    if not smooth:
        smoothed.set_data(0, 0)
        smoothed.figure.canvas.draw()
        
       
viewer.window.add_dock_widget(GetVolume, area='right')
